In [1]:
import pandas as pd

In [2]:
from glob import glob 

In [3]:
print("Bem Vindo(a) :D ")

Bem Vindo(a) :D 


In [4]:
print("Lendo bases...")

Lendo bases...


### Leitura das bases

In [5]:
df_base_atestados = pd.read_excel(glob("bases/base_ambulatorio/*")[0], index_col = None)

In [6]:
df_base_lojas_centro = pd.read_excel(glob("bases/base_nomes_lojas/*")[0], index_col = None)

In [7]:
df_dicionario_funcoes = pd.read_excel(glob("bases/base_nomes_funcao/*")[0], index_col = None)

In [8]:
df_dicionario_funcoes.head()

,funcao,codigo
0,ANALISTA DE PEDIDO JR,1
1,ANALISTA DE PEDIDOS PL,2
2,APRENDIZ,3
3,ASSIST. ADMINISTRATIVO,4
4,ASSIST. DE CRIACAO DE CONTEUDO,5


In [9]:
df_base_lojas_copiado = df_base_lojas_centro.copy()

In [10]:
df_base_lojas_genero = df_base_lojas_centro.copy()

In [11]:
df_base_atestados.columns = ['data_sap','matricula','nome','idade','sexo','funcao','codigo_funcao','loja','centro','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc','','','']

In [12]:
df_base_atestados_filtrado = df_base_atestados[['data_sap','matricula','nome','idade','sexo','funcao','codigo_funcao','loja','centro','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc']].copy()

In [13]:
df_base_atestados_filtrado = df_base_atestados_filtrado[~df_base_atestados_filtrado['matricula'].isnull()].copy()

In [14]:
df_base_atestados_filtrado = df_base_atestados_filtrado[df_base_atestados_filtrado['nome'] != 'NOME'].copy()

In [15]:
df_base_atestados_filtrado.columns 

Index(['data_sap', 'matricula', 'nome', 'idade', 'sexo', 'funcao',
       'codigo_funcao', 'loja', 'centro', 'data_inicial', 'data_final',
       'vai_inss', 'dias_perdidos', 'cib_10', 'grupo_doenca', 'lcdo_soc'],
      dtype='object')

#### filtro das colunas importantes

In [16]:
df_atestados_meses = df_base_atestados_filtrado[['data_inicial','dias_perdidos','loja','centro','sexo','idade','funcao','codigo_funcao','cib_10','grupo_doenca']].copy()

### Afirmando que os nomes das lojas estarão em caixa alta

In [17]:
df_atestados_meses['loja'] = df_atestados_meses['loja'].astype(str)

In [18]:
df_atestados_meses['loja'] = df_atestados_meses['loja'].str.upper() 

In [19]:
### Pegando somente o dado que necessita que é o mês
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].astype(str)

In [20]:
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(".","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(":","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(" ","-")

In [21]:
df_atestados_meses['mes'] = df_atestados_meses['data_inicial'].apply(lambda item: str(item).split('-')[1])

#  SEPARAÇÃO DOS DATA FRAMES POR FILTRO ESPECÍFICO DE CADA SITUAÇÃO 


## ------ Filtro para Gerar Quantidade de licenças médicas por Unidade

In [22]:
df_licencas_unidade = df_atestados_meses[['mes','centro','dias_perdidos']].copy()

In [23]:
df_licencas_grupo_doenca = df_atestados_meses[['mes','cib_10','grupo_doenca']].copy()

In [24]:
df_faixa_etaria = df_atestados_meses[['mes','idade']].copy()

In [25]:
df_bases_funcoes = df_atestados_meses[['mes','codigo_funcao']].copy()

In [26]:
df_licencas_genero = df_atestados_meses[['mes','centro','sexo']].copy()

In [27]:
df_dias_perdidos_unidade = df_licencas_unidade.copy()

In [28]:
df_atestados_meses.drop(columns = {'data_inicial'}, inplace = True)

In [29]:
df_atestados_meses['mes'] = df_atestados_meses['mes'].astype(int)

In [30]:
df_atestados_meses['mes_copiado'] = df_atestados_meses['mes'].copy()

## QUANTIDADE DE DIAS PERDIDOS E LICENÇAS POR MÊS

In [31]:
df_atestados_meses = df_atestados_meses.groupby('mes', as_index = False).agg({'mes_copiado': 'count','dias_perdidos':'sum'})

In [32]:
df_atestados_meses

,mes,mes_copiado,dias_perdidos
0,5,149,353
1,6,162,341
2,7,200,485
3,8,199,426
4,9,153,332
5,10,199,411


In [33]:
df_atestados_meses.rename(columns = {'mes_copiado':'QTE DE LICENÇAS MÉDICAS','dias_perdidos':'QTE DIAS PERDIDOS'}, inplace = True)

In [34]:
nome_mes = {1: 'JAN', 2: 'FEV', 3:'MARC', 4:'ABR', 5:'MAIO', 6:'JUN', 7:'JUL', 8:'AGO', 9:'SET', 10:'OUT', 11:'NOV' , 12: 'DEZ'}

In [35]:
df_atestados_meses['nome_mes'] = df_atestados_meses['mes'].map(nome_mes)

In [36]:
df_atestados_meses

,mes,QTE DE LICENÇAS MÉDICAS,QTE DIAS PERDIDOS,nome_mes
0,5,149,353,MAIO
1,6,162,341,JUN
2,7,200,485,JUL
3,8,199,426,AGO
4,9,153,332,SET
5,10,199,411,OUT


In [37]:
df_atestados_meses.sort_values(by = ['mes'], ascending = True, inplace = True)

In [38]:
df_atestados_meses

,mes,QTE DE LICENÇAS MÉDICAS,QTE DIAS PERDIDOS,nome_mes
0,5,149,353,MAIO
1,6,162,341,JUN
2,7,200,485,JUL
3,8,199,426,AGO
4,9,153,332,SET
5,10,199,411,OUT


In [39]:
df_atestados_meses = df_atestados_meses.T 

In [40]:
df_atestados_meses

,0,1,2,3,4,5
mes,5,6,7,8,9,10
QTE DE LICENÇAS MÉDICAS,149,162,200,199,153,199
QTE DIAS PERDIDOS,353,341,485,426,332,411
nome_mes,MAIO,JUN,JUL,AGO,SET,OUT


In [41]:
#Reorganizando os meses com os respectivos nomes
df_atestados_meses.columns = df_atestados_meses.loc['nome_mes'] 
df_atestados_meses.columns.name = None

In [42]:
df_atestados_meses

,MAIO,JUN,JUL,AGO,SET,OUT
mes,5,6,7,8,9,10
QTE DE LICENÇAS MÉDICAS,149,162,200,199,153,199
QTE DIAS PERDIDOS,353,341,485,426,332,411
nome_mes,MAIO,JUN,JUL,AGO,SET,OUT


In [43]:
df_atestados_meses.drop(['mes', 'nome_mes'], inplace = True)

In [44]:
df_atestados_meses

,MAIO,JUN,JUL,AGO,SET,OUT
QTE DE LICENÇAS MÉDICAS,149,162,200,199,153,199
QTE DIAS PERDIDOS,353,341,485,426,332,411


In [45]:
lista_colunas = df_atestados_meses.columns

lista_meses = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

lista_meses = [x for x in lista_meses if x not in lista_colunas] 

In [46]:
lista_meses

['JAN', 'FEV', 'MAR', 'ABR', 'NOV', 'DEZ']

In [47]:
for mes in lista_meses:
    df_atestados_meses[mes] = 0

In [48]:
df_atestados_meses.rename(columns = {'JAN': 1,"FEV":2,"MAR":3,"ABR":4,"MAIO":5,"JUN":6,"JUL":7,"AGO":8,"SET":9,"OUT":10,"NOV":11,"DEZ":12}, inplace = True)

In [49]:
df_atestados_meses.sort_index(axis = 1, inplace = True)

In [50]:
df_atestados_meses.rename(columns = {1: 'JANEIRO',2:"FEV",3:"MAR",4:"ABR",5:"MAIO",6:"JUN",7:"JUL",8:"AGO",9:"SET",10:"OUT",11:"NOV",12:"DEZ"}, inplace = True)

In [51]:
df_atestados_meses

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
QTE DE LICENÇAS MÉDICAS,0,0,0,0,149,162,200,199,153,199,0,0
QTE DIAS PERDIDOS,0,0,0,0,353,341,485,426,332,411,0,0


## QUANTIDADE DE LICENÇAS MÉDICAS POR UNIDADE

### Função que tem por finalidade dizer onde tem traço e trazer so a segunda posição, ou seja, restando somente o nome da loja

In [52]:
# def consertando(row):
#     if '-' in row['loja']:
#         row['loja'] = row['loja'].split('-')[1]
#     return row

# df_licencas_unidade = df_licencas_unidade.apply(consertando, axis = 1)

## Contando pelo centro a quantidade de licenças médicas por unidade

## data frame para renomear as lojas

In [53]:
df_base_lojas_centro.shape

(44, 2)

In [54]:
df_base_lojas_centro.head()

,loja,centro
0,ARIQUEMES,206
1,AVENIDA,103
2,BARROSO,107
3,BEMOL ONLINE,102
4,CAMAPUÃ,118


In [55]:
df_licencas_unidade['centro'] = df_licencas_unidade['centro'].astype(int)

In [56]:
df_licencas_unidade['mes_unidades_copiado'] = df_licencas_unidade['mes'].copy()

In [57]:
df_licencas_unidade['mes'] = df_licencas_unidade['mes'].astype(int)

In [58]:
df_licencas_unidade.head()

,mes,centro,dias_perdidos,mes_unidades_copiado
0,5,107,1,05
1,5,114,1,05
2,5,201,7,05
3,5,201,2,05
4,5,202,2,05


In [59]:
lista_ordenacao = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k','l']

In [60]:
df_licencas_unidade

,mes,centro,dias_perdidos,mes_unidades_copiado
0,5,107,1,05
1,5,114,1,05
2,5,201,7,05
3,5,201,2,05
4,5,202,2,05
5,5,701,3,05
6,5,105,15,05
7,5,500,1,05
8,5,118,2,05
9,5,118,2,05


In [61]:
for mes in df_licencas_unidade['mes'].unique():
    
    df_aux = df_licencas_unidade[df_licencas_unidade['mes'] == mes].copy()
   
    df_agrupado = df_aux.groupby(['centro'], as_index = False).agg({'mes_unidades_copiado': 'count'}).copy()
       
    df_agrupado.rename(columns={"mes_unidades_copiado": lista_ordenacao[mes]}, inplace = True)
    
    df_base_lojas_centro = pd.merge(df_base_lojas_centro, df_agrupado, on = ['centro'], how = 'outer')
    
    df_base_lojas_centro.fillna(0, inplace = True)

In [62]:
df_base_lojas_centro.head()

,loja,centro,f,g,h,i,j,k
0,ARIQUEMES,206,0.0,0.0,0.0,0.0,0.0,0.0
1,AVENIDA,103,6.0,8.0,9.0,15.0,5.0,15.0
2,BARROSO,107,11.0,4.0,9.0,2.0,2.0,4.0
3,BEMOL ONLINE,102,0.0,4.0,3.0,3.0,1.0,2.0
4,CAMAPUÃ,118,8.0,10.0,10.0,19.0,18.0,13.0


In [63]:
df_base_lojas_centro.rename(columns = {"centro":"a_centro", "loja":'a_loja'}, inplace = True)

In [64]:
df_base_lojas_centro

,a_loja,a_centro,f,g,h,i,j,k
0,ARIQUEMES,206,0.0,0.0,0.0,0.0,0.0,0.0
1,AVENIDA,103,6.0,8.0,9.0,15.0,5.0,15.0
2,BARROSO,107,11.0,4.0,9.0,2.0,2.0,4.0
3,BEMOL ONLINE,102,0.0,4.0,3.0,3.0,1.0,2.0
4,CAMAPUÃ,118,8.0,10.0,10.0,19.0,18.0,13.0
5,CIDADE NOVA,115,11.0,5.0,17.0,13.0,9.0,20.0
6,DB PONTA NEGRA,114,8.0,12.0,5.0,8.0,0.0,7.0
7,EDUCANDOS,105,9.0,7.0,6.0,11.0,6.0,5.0
8,GRANDE CIRCULAR,109,5.0,5.0,10.0,13.0,8.0,13.0
9,ITACOATIARA,510,1.0,1.0,7.0,4.0,8.0,6.0


In [65]:
df_base_lojas_centro.head()

,a_loja,a_centro,f,g,h,i,j,k
0,ARIQUEMES,206,0.0,0.0,0.0,0.0,0.0,0.0
1,AVENIDA,103,6.0,8.0,9.0,15.0,5.0,15.0
2,BARROSO,107,11.0,4.0,9.0,2.0,2.0,4.0
3,BEMOL ONLINE,102,0.0,4.0,3.0,3.0,1.0,2.0
4,CAMAPUÃ,118,8.0,10.0,10.0,19.0,18.0,13.0


In [66]:
df_base_lojas_centro.sort_index(axis = 1, inplace = True)

In [67]:
dic_mes_alfabeto = {'a': 'JAN', 'b': 'FEV', 'c': 'MAR', 'd': 'ABR', 'e':'MAIO', 'f':'JUN', 'g':'JUL', 'h':'AGO', 'i':'SET', 'j':'OUT', 'k':'NOV','l':'DEZ'}

In [68]:
df_base_lojas_centro.head()

,a_centro,a_loja,f,g,h,i,j,k
0,206,ARIQUEMES,0.0,0.0,0.0,0.0,0.0,0.0
1,103,AVENIDA,6.0,8.0,9.0,15.0,5.0,15.0
2,107,BARROSO,11.0,4.0,9.0,2.0,2.0,4.0
3,102,BEMOL ONLINE,0.0,4.0,3.0,3.0,1.0,2.0
4,118,CAMAPUÃ,8.0,10.0,10.0,19.0,18.0,13.0


In [69]:
for name in df_base_lojas_centro.columns[2:]:
    df_base_lojas_centro.rename(columns = {name: dic_mes_alfabeto[name]},inplace = True)

In [70]:
df_base_lojas_centro.head()

,a_centro,a_loja,JUN,JUL,AGO,SET,OUT,NOV
0,206,ARIQUEMES,0.0,0.0,0.0,0.0,0.0,0.0
1,103,AVENIDA,6.0,8.0,9.0,15.0,5.0,15.0
2,107,BARROSO,11.0,4.0,9.0,2.0,2.0,4.0
3,102,BEMOL ONLINE,0.0,4.0,3.0,3.0,1.0,2.0
4,118,CAMAPUÃ,8.0,10.0,10.0,19.0,18.0,13.0


In [71]:
df_base_lojas_centro.columns

Index(['a_centro', 'a_loja', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV'], dtype='object')

In [72]:
lista_colunas = df_base_lojas_centro.columns

lista_semestre = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

if "JUL" in lista_colunas:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]
else:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]

In [73]:
lista_meses

['JAN', 'FEV', 'MAR', 'ABR', 'MAIO', 'DEZ']

In [74]:
for mes in lista_meses:
    df_base_lojas_centro[mes] = 0

In [75]:
df_base_lojas_centro = df_base_lojas_centro[["a_loja","JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]].copy()

In [76]:
df_base_lojas_centro.rename(columns = {'a_loja':'LOJA'}, inplace = True)

In [77]:
df_qtd_atestados_unidade = df_base_lojas_centro.copy()

##  QUANTIDADE DE DIAS PERDIDOS POR MÊS

In [78]:
df_dias_perdidos_unidade['mes'] = df_dias_perdidos_unidade['mes'].astype(int)

In [79]:
df_dias_perdidos_unidade['centro'] = df_dias_perdidos_unidade['centro'].astype(int)

In [80]:
for mes in df_dias_perdidos_unidade['mes'].unique():
    
    df_aux = df_dias_perdidos_unidade[df_dias_perdidos_unidade['mes'] == mes].copy()
   
    df_agrupando = df_aux.groupby(['centro'], as_index = False).agg({'dias_perdidos': 'sum'}).copy()
       
    df_agrupando.rename(columns = {"dias_perdidos": lista_ordenacao[mes]}, inplace = True)
    
    df_base_lojas_copiado = pd.merge(df_base_lojas_copiado, df_agrupando, on = ['centro'], how = 'outer')
    
    df_base_lojas_copiado.fillna(0, inplace = True)

In [81]:
# df_base_lojas_copiado.tail()

In [82]:
df_base_lojas_copiado.rename(columns = {"centro":"a_centro", "loja":'a_loja'}, inplace = True)

In [83]:
df_base_lojas_copiado.sort_index(axis = 1, inplace = True)

In [84]:
for name in df_base_lojas_copiado.columns[2:]:
    df_base_lojas_copiado.rename(columns = {name: dic_mes_alfabeto[name]},inplace = True)

In [85]:
lista_colunas = df_base_lojas_copiado.columns

lista_semestre = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

if "JUL" in lista_colunas:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]
else:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]

In [86]:
for mes in lista_meses:
    df_base_lojas_copiado[mes] = 0

In [87]:
df_base_lojas_copiado = df_base_lojas_copiado[["a_loja","JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]].copy()

In [88]:
df_base_lojas_centro.rename(columns = {'a_loja':'LOJA'}, inplace = True)

In [89]:
df_qtd_dias_perdidos_unidade = df_base_lojas_centro.copy()

## QUANTITATIVO DE LICENÇAS MÉDICAS POR GÊNERO 

In [90]:
df_licencas_genero['centro'] = df_licencas_genero['centro'].astype(int)

In [91]:
df_licencas_genero['mes'] = df_licencas_genero['mes'].astype(int)

In [92]:
## deixandos todos em caixa alta
df_licencas_genero['sexo'] = df_licencas_genero['sexo'].str.upper()

In [93]:
df_licencas_genero.drop(columns = {"centro"}, inplace  = True)

In [94]:
df_feminino = df_licencas_genero[df_licencas_genero['sexo'] == 'F'].copy()

In [95]:
df_masculino = df_licencas_genero[df_licencas_genero['sexo'] == 'M'].copy()

In [96]:
df_feminino = df_feminino.groupby('mes', as_index = False).agg({'sexo': 'count'})

In [97]:
############################## CONTAGEM DOS FEMININOS #################

In [98]:
nome_mes_genero = {1: 'JAN', 2: 'FEV', 3:'MARC', 4:'ABR', 5:'MAIO', 6:'JUN', 7:'JUL', 8:'AGO', 9:'SET', 10:'OUT', 11:'NOV' , 12: 'DEZ'}

In [99]:
df_feminino

,mes,sexo
0,5,120
1,6,116
2,7,130
3,8,138
4,9,109
5,10,120


In [100]:
df_feminino['nome_mes_genero'] = df_feminino['mes'].map(nome_mes_genero)

In [101]:
df_feminino

,mes,sexo,nome_mes_genero
0,5,120,MAIO
1,6,116,JUN
2,7,130,JUL
3,8,138,AGO
4,9,109,SET
5,10,120,OUT


In [102]:
df_feminino.rename(columns = {"sexo":"FEMININO"}, inplace = True)

In [103]:
df_feminino.sort_values(by = ['mes'], ascending = True, inplace = True)

In [104]:
df_feminino = df_feminino.T 

In [105]:
df_feminino

,0,1,2,3,4,5
mes,5,6,7,8,9,10
FEMININO,120,116,130,138,109,120
nome_mes_genero,MAIO,JUN,JUL,AGO,SET,OUT


In [106]:
#Reorganizando os meses com os respectivos nomes
df_feminino.columns = df_feminino.loc['nome_mes_genero'] 
df_feminino.columns.name = None

In [107]:
df_feminino

,MAIO,JUN,JUL,AGO,SET,OUT
mes,5,6,7,8,9,10
FEMININO,120,116,130,138,109,120
nome_mes_genero,MAIO,JUN,JUL,AGO,SET,OUT


In [108]:
df_feminino.drop(['mes', 'nome_mes_genero'], inplace = True)

In [109]:
df_feminino

,MAIO,JUN,JUL,AGO,SET,OUT
FEMININO,120,116,130,138,109,120


In [110]:
lista_colunas_genero = df_feminino.columns

lista_meses_genero = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

lista_meses_genero = [x for x in lista_meses_genero if x not in lista_colunas_genero] 

In [111]:
lista_meses_genero

['JAN', 'FEV', 'MAR', 'ABR', 'NOV', 'DEZ']

In [112]:
for mes in lista_meses_genero:
    df_feminino[mes] = 0

In [113]:
df_feminino

,MAIO,JUN,JUL,AGO,SET,OUT,JAN,FEV,MAR,ABR,NOV,DEZ
FEMININO,120,116,130,138,109,120,0,0,0,0,0,0


In [114]:
df_feminino.rename(columns = {'JAN': 1,"FEV":2,"MAR":3,"ABR":4,"MAIO":5,"JUN":6,"JUL":7,"AGO":8,"SET":9,"OUT":10,"NOV":11,"DEZ":12}, inplace = True)

In [115]:
df_feminino.sort_index(axis = 1, inplace = True)

In [116]:
df_feminino.rename(columns = {1: 'JANEIRO',2:"FEV",3:"MAR",4:"ABR",5:"MAIO",6:"JUN",7:"JUL",8:"AGO",9:"SET",10:"OUT",11:"NOV",12:"DEZ"}, inplace = True)

In [117]:
df_feminino

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
FEMININO,0,0,0,0,120,116,130,138,109,120,0,0


In [118]:
#################### Inicio da contagem dos MASCULINOS ##############################

In [119]:
df_masculino = df_masculino.groupby('mes', as_index = False).agg({'sexo': 'count'})

In [120]:
df_masculino

,mes,sexo
0,5,29
1,6,46
2,7,70
3,8,61
4,9,44
5,10,79


In [121]:
df_masculino['nome_mes_genero'] = df_masculino['mes'].map(nome_mes_genero)

In [122]:
df_masculino

,mes,sexo,nome_mes_genero
0,5,29,MAIO
1,6,46,JUN
2,7,70,JUL
3,8,61,AGO
4,9,44,SET
5,10,79,OUT


In [123]:
df_masculino.rename(columns = {"sexo":"MASCULINO"}, inplace = True)

In [124]:
df_masculino.sort_values(by = ['mes'], ascending = True, inplace = True)

In [125]:
df_masculino = df_masculino.T 

In [126]:
df_masculino

,0,1,2,3,4,5
mes,5,6,7,8,9,10
MASCULINO,29,46,70,61,44,79
nome_mes_genero,MAIO,JUN,JUL,AGO,SET,OUT


In [127]:
#Reorganizando os meses com os respectivos nomes
df_masculino.columns = df_masculino.loc['nome_mes_genero'] 
df_masculino.columns.name = None

In [128]:
df_masculino

,MAIO,JUN,JUL,AGO,SET,OUT
mes,5,6,7,8,9,10
MASCULINO,29,46,70,61,44,79
nome_mes_genero,MAIO,JUN,JUL,AGO,SET,OUT


In [129]:
df_masculino.drop(['mes', 'nome_mes_genero'], inplace = True)

In [130]:
df_masculino

,MAIO,JUN,JUL,AGO,SET,OUT
MASCULINO,29,46,70,61,44,79


In [131]:
lista_colunas_genero = df_masculino.columns

lista_meses_genero = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

lista_meses_genero = [x for x in lista_meses_genero if x not in lista_colunas_genero] 

In [132]:
for mes in lista_meses_genero:
    df_masculino[mes] = 0

In [133]:
df_masculino

,MAIO,JUN,JUL,AGO,SET,OUT,JAN,FEV,MAR,ABR,NOV,DEZ
MASCULINO,29,46,70,61,44,79,0,0,0,0,0,0


In [134]:
df_masculino.rename(columns = {'JAN': 1,"FEV":2,"MAR":3,"ABR":4,"MAIO":5,"JUN":6,"JUL":7,"AGO":8,"SET":9,"OUT":10,"NOV":11,"DEZ":12}, inplace = True)

In [135]:
df_masculino

,5,6,7,8,9,10,1,2,3,4,11,12
MASCULINO,29,46,70,61,44,79,0,0,0,0,0,0


In [136]:
df_masculino.sort_index(axis = 1, inplace = True)

In [137]:
df_masculino

,1,2,3,4,5,6,7,8,9,10,11,12
MASCULINO,0,0,0,0,29,46,70,61,44,79,0,0


In [138]:
df_masculino.rename(columns = {1: 'JANEIRO',2:"FEV",3:"MAR",4:"ABR",5:"MAIO",6:"JUN",7:"JUL",8:"AGO",9:"SET",10:"OUT",11:"NOV",12:"DEZ"}, inplace = True)

In [139]:
df_masculino

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
MASCULINO,0,0,0,0,29,46,70,61,44,79,0,0


In [140]:
df_final_genero = pd.concat([df_feminino,df_masculino])

In [141]:
df_final_genero

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
FEMININO,0,0,0,0,120,116,130,138,109,120,0,0
MASCULINO,0,0,0,0,29,46,70,61,44,79,0,0


## Quantidade de Licenças médicas por intervalo de idade

In [142]:
df_faixa_etaria.head()

,mes,idade
0,05,55
1,05,31
2,05,30
3,05,30
4,05,27


In [143]:
df_faixa_etaria['idade'] = df_faixa_etaria['idade'].astype(int)
df_faixa_etaria['mes'] = df_faixa_etaria['mes'].astype(int)

In [144]:
# def faixa_etaria(idade):
#     if idade<=29:
#         return 'idade_16_29'
#     elif idade>=30 and idade<=39:
#         return 'idade_30_39'
#     elif idade>=40 and idade<=49:
#         return 'idade_40_49'
#     elif idade>=50 and idade<=59:
#         return 'idade_50_59'
#     elif idade>=60:
#         return 'idade_maior_60'

In [145]:
# df_faixa_etaria['faixa'] = df_faixa_etaria['idade'].apply(faixa_etaria)

In [146]:
# a.groupby(['faixa', 'mes'])['idade'].count().reset_index()

In [147]:
for mes in df_faixa_etaria['mes'].unique(): 
    
    df_aux = df_faixa_etaria[df_faixa_etaria['mes'] == mes].copy()
    
    df_16_29 = df_faixa_etaria[(df_faixa_etaria['idade']<=29)]
    df_16_29 = df_16_29.groupby(['mes'], as_index = False).agg({'idade': 'count'})
    
    df_30_39 = df_faixa_etaria[(df_faixa_etaria['idade']>=30) & (df_faixa_etaria['idade']<=39)]
    df_30_39 = df_30_39.groupby(['mes'], as_index = False).agg({'idade': 'count'})
    
    df_40_49 = df_faixa_etaria[(df_faixa_etaria['idade']>=40) & (df_faixa_etaria['idade']<=49)]
    df_40_49 = df_40_49.groupby(['mes'], as_index = False).agg({'idade': 'count'})
        
    df_50_59 = df_faixa_etaria[(df_faixa_etaria['idade']>=50) & (df_faixa_etaria['idade']<=59)]
    df_50_59 = df_50_59.groupby(['mes'], as_index = False).agg({'idade': 'count'})
    
    df_acima_60 = df_faixa_etaria[(df_faixa_etaria['idade']>=60)]
    df_acima_60 = df_acima_60.groupby(['mes'], as_index = False).agg({'idade': 'count'})

### CODIGO DO LEO QUE SEPARA AS FAIXAS E PELO MES

In [148]:
# df = pd.DataFrame()

# for faixa in faixas:
#         aux = df_faixa_etaria[df_faixa_etaria['faixa'] == faixa].copy()
#         aux = aux.groupby(['faixa', 'mes'])['idade'].count().reset_index()
#         df = pd.concat([df,aux])
#         df['mes'] = df['mes'].astype(str)
#         df_teste = df[df['faixa'] == faixa]
#         df = df.astype(str)    
#         df_final_etaria = pd.merge(df_meses,df, on = 'mes', how = 'outer')

### Renomeando as colunas

In [149]:
df_16_29.rename(columns = {"idade":"16 A 29 ANOS"}, inplace = True)
df_30_39.rename(columns = {"idade":"30 A 39 ANOS"}, inplace = True)
df_40_49.rename(columns = {"idade":"40 A 49 ANOS"}, inplace = True)
df_50_59.rename(columns = {"idade":"50 A 59 ANOS"}, inplace = True)
df_acima_60.rename(columns = {"idade":"ACIMA DE 60 ANOS"}, inplace = True)

In [150]:
list_meses = {'mes': [1,2,3,4,5,6,7,8,9,10,11,12]}

df_meses = pd.DataFrame(list_meses)

In [151]:
df_final_intervalos_idade = pd.merge(df_16_29,df_meses, on = 'mes', how = 'outer')

In [152]:
df_final_intervalos_idade.sort_values(by = ['mes'], ascending = True, inplace = True)

In [153]:
df_final_intervalos_idade = pd.merge(df_final_intervalos_idade, df_30_39, on  = 'mes', how = 'outer')

In [154]:
df_final_intervalos_idade = pd.merge(df_final_intervalos_idade, df_40_49, on  = 'mes', how = 'outer')

In [155]:
df_final_intervalos_idade = pd.merge(df_final_intervalos_idade, df_50_59, on  = 'mes', how = 'outer')

In [156]:
df_final_intervalos_idade = pd.merge(df_final_intervalos_idade, df_acima_60, on  = 'mes', how = 'outer')

In [157]:
df_final_intervalos_idade.fillna(0, inplace = True)

In [158]:
df_final_intervalos_idade = df_final_intervalos_idade.astype(int)

In [159]:
df_final_intervalos_idade['nome_mes_genero'] = df_final_intervalos_idade['mes'].map(nome_mes_genero)

In [160]:
df_final_intervalos_idade = df_final_intervalos_idade.T

In [161]:
df_final_intervalos_idade

,0,1,2,3,4,5,6,7,8,9,10,11
mes,1,2,3,4,5,6,7,8,9,10,11,12
16 A 29 ANOS,0,0,0,0,78,77,104,95,88,92,0,0
30 A 39 ANOS,0,0,0,0,61,69,76,87,48,80,0,0
40 A 49 ANOS,0,0,0,0,8,14,16,16,14,20,0,0
50 A 59 ANOS,0,0,0,0,2,1,3,1,2,6,0,0
ACIMA DE 60 ANOS,0,0,0,0,0,1,1,0,1,1,0,0
nome_mes_genero,JAN,FEV,MARC,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ


In [162]:
#Reorganizando os meses com os respectivos nomes
df_final_intervalos_idade.columns = df_final_intervalos_idade.loc['nome_mes_genero'] 
df_final_intervalos_idade.columns.name = None

In [163]:
df_final_intervalos_idade.drop(['mes', 'nome_mes_genero'], inplace = True)

In [164]:
df_final_intervalos_idade

,JAN,FEV,MARC,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
16 A 29 ANOS,0,0,0,0,78,77,104,95,88,92,0,0
30 A 39 ANOS,0,0,0,0,61,69,76,87,48,80,0,0
40 A 49 ANOS,0,0,0,0,8,14,16,16,14,20,0,0
50 A 59 ANOS,0,0,0,0,2,1,3,1,2,6,0,0
ACIMA DE 60 ANOS,0,0,0,0,0,1,1,0,1,1,0,0


## GERANDO TABELA DE QTD DE LICENÇAS POR FUNÇÃO

In [165]:
df_bases_funcoes['codigo_funcao'] = df_bases_funcoes['codigo_funcao'].astype(int)

In [166]:
df_bases_funcoes['mes_copiado'] = df_bases_funcoes['mes'].copy()

In [167]:
df_bases_funcoes['mes'] = df_bases_funcoes['mes'].astype(int)
df_bases_funcoes['mes_copiado'] = df_bases_funcoes['mes_copiado'].astype(int)

In [168]:
df_bases_funcoes.dtypes

mes              int64
codigo_funcao    int64
mes_copiado      int64
dtype: object

In [169]:
df_bases_funcoes.head(1)

,mes,codigo_funcao,mes_copiado
0,5,49,5


In [170]:
df_dicionario_funcoes.rename(columns = {'codigo':'codigo_funcao'}, inplace = True)

In [171]:
df_dicionario_funcoes.head()

,funcao,codigo_funcao
0,ANALISTA DE PEDIDO JR,1
1,ANALISTA DE PEDIDOS PL,2
2,APRENDIZ,3
3,ASSIST. ADMINISTRATIVO,4
4,ASSIST. DE CRIACAO DE CONTEUDO,5


### fazendo merge pelo codigo funcao e contando quantos atestados tem no mes

In [172]:
lista_ordenacao_func = ['','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k','l']

In [173]:
df_dicionario_funcoes.head(1)

,funcao,codigo_funcao
0,ANALISTA DE PEDIDO JR,1


In [174]:
for mes in df_bases_funcoes['mes'].unique():
    
    df_aux = df_bases_funcoes[df_bases_funcoes['mes'] == mes].copy()
   
    df_agrupado = df_aux.groupby(['codigo_funcao'], as_index = False).agg({'mes_copiado': 'count'}).copy()
       
    df_agrupado.rename(columns={"mes_copiado": lista_ordenacao_func[mes]}, inplace = True)
    
    df_dicionario_funcoes = pd.merge(df_dicionario_funcoes, df_agrupado, on = ['codigo_funcao'], how = 'outer')
    
    df_dicionario_funcoes.fillna(0, inplace = True)

In [175]:
df_dicionario_funcoes.head() 

,funcao,codigo_funcao,e,f,g,h,i,j
0,ANALISTA DE PEDIDO JR,1,0.0,0.0,0.0,0.0,1.0,2.0
1,ANALISTA DE PEDIDOS PL,2,0.0,1.0,1.0,1.0,0.0,0.0
2,APRENDIZ,3,4.0,10.0,5.0,12.0,5.0,7.0
3,ASSIST. ADMINISTRATIVO,4,0.0,0.0,0.0,2.0,0.0,0.0
4,ASSIST. DE CRIACAO DE CONTEUDO,5,0.0,0.0,1.0,0.0,0.0,1.0


In [176]:
df_dicionario_funcoes.rename(columns = {"funcao":"a_funcao", "codigo_funcao":'a_codigo_funcao'}, inplace = True)

In [177]:
df_dicionario_funcoes.sort_index(axis = 1, inplace = True)

In [178]:
for name in df_dicionario_funcoes.columns[2:]:
    df_dicionario_funcoes.rename(columns = {name: dic_mes_alfabeto[name]},inplace = True)

In [179]:
lista_colunas_func = df_dicionario_funcoes.columns

In [180]:
lista_semestre_func = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

In [181]:
lista_semestre = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

if "JUL" in lista_colunas_func:
    lista_meses = [x for x in lista_semestre_func if x not in lista_colunas_func]
else:
    lista_meses = [x for x in lista_semestre_func if x not in lista_colunas_func]

In [182]:
for mes in lista_meses:
    df_dicionario_funcoes[mes] = 0

In [183]:
df_dicionario_funcoes.head()

,a_codigo_funcao,a_funcao,MAIO,JUN,JUL,AGO,SET,OUT,JAN,FEV,MAR,ABR,NOV,DEZ
0,1,ANALISTA DE PEDIDO JR,0.0,0.0,0.0,0.0,1.0,2.0,0,0,0,0,0,0
1,2,ANALISTA DE PEDIDOS PL,0.0,1.0,1.0,1.0,0.0,0.0,0,0,0,0,0,0
2,3,APRENDIZ,4.0,10.0,5.0,12.0,5.0,7.0,0,0,0,0,0,0
3,4,ASSIST. ADMINISTRATIVO,0.0,0.0,0.0,2.0,0.0,0.0,0,0,0,0,0,0
4,5,ASSIST. DE CRIACAO DE CONTEUDO,0.0,0.0,1.0,0.0,0.0,1.0,0,0,0,0,0,0


In [184]:
df_dicionario_funcoes = df_dicionario_funcoes[["a_funcao","JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]].copy()

In [185]:
df_dicionario_funcoes.rename(columns = {'a_funcao':'FUNÇÃO'}, inplace = True)

In [186]:
df_dicionario_funcoes['JAN'] = df_dicionario_funcoes['JAN'].astype(int)
df_dicionario_funcoes['FEV'] = df_dicionario_funcoes['FEV'].astype(int)
df_dicionario_funcoes['MAR'] = df_dicionario_funcoes['MAR'].astype(int)
df_dicionario_funcoes['ABR'] = df_dicionario_funcoes['ABR'].astype(int)
df_dicionario_funcoes['MAIO'] = df_dicionario_funcoes['MAIO'].astype(int)
df_dicionario_funcoes['JUN'] = df_dicionario_funcoes['JUN'].astype(int)
df_dicionario_funcoes['JUL'] = df_dicionario_funcoes['JUL'].astype(int)
df_dicionario_funcoes['AGO'] = df_dicionario_funcoes['AGO'].astype(int)
df_dicionario_funcoes['SET'] = df_dicionario_funcoes['SET'].astype(int)
df_dicionario_funcoes['OUT'] = df_dicionario_funcoes['OUT'].astype(int)
df_dicionario_funcoes['NOV'] = df_dicionario_funcoes['NOV'].astype(int)
df_dicionario_funcoes['DEZ'] = df_dicionario_funcoes['DEZ'].astype(int)

In [187]:
df_dicionario_funcoes.head()

,FUNÇÃO,JAN,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,ANALISTA DE PEDIDO JR,0,0,0,0,0,0,0,0,1,2,0,0
1,ANALISTA DE PEDIDOS PL,0,0,0,0,0,1,1,1,0,0,0,0
2,APRENDIZ,0,0,0,0,4,10,5,12,5,7,0,0
3,ASSIST. ADMINISTRATIVO,0,0,0,0,0,0,0,2,0,0,0,0
4,ASSIST. DE CRIACAO DE CONTEUDO,0,0,0,0,0,0,1,0,0,1,0,0


# Quantitativo de licenças médicas por grupo de doença

In [188]:
df_licencas_grupo_doenca = df_licencas_grupo_doenca[(~df_licencas_grupo_doenca['cib_10'].str.contains('CID-10'))].copy()

### FUNÇÃO PARA COLOCAR COMO "OUTROS"

In [189]:
df_licencas_grupo_doenca.head()

,mes,cib_10,grupo_doenca
0,05,Z019,EXAME DE VISÃO
1,05,Z32.1,OBSTÉTRICO
2,05,J45,RESPIRATÓRIO
3,05,Z76.3,ACOMPANHANTE
4,05,R05,RESPIRATÓRIO


In [190]:
# (row['grupo_doenca'] != 'GASTROINTESTINAL') || 
#         (row['grupo_doenca'] != 'ACOMPANHANTE') ||
#         (row['grupo_doenca'] != 'OBSTÉTRICA') || (row['grupo_doenca'] != 'ODONTOLÓGICA') || 
#         (row['grupo_doenca'] != 'OSTEOMUSCULAR') || (row['grupo_doenca'] != 'PSIQUIÁTRICA')
#         || (row['grupo_doenca'] != 'SEM CID-10'))

In [195]:
def colocaComoOutros(row):
    
    if (row['grupo_doenca'] != 'RESPIRATÓRIO'):
        row['grupo_doenca'] = 'OUTRAS CAUSAS'
    else:
        if (row['grupo_doenca'] != 'ACOMPANHANTE'):
        row['grupo_doenca'] = 'OUTRAS CAUSAS'
#     else:
#         if (row['grupo_doenca'] != 'OBSTÉTRICA'):
#         row['grupo_doenca'] = 'OUTRAS CAUSAS'
#     else:
#         if (row['grupo_doenca'] != 'ODONTOLÓGICA'):
#         row['grupo_doenca'] = 'OUTRAS CAUSAS'
#     else:
#         if (row['grupo_doenca'] != 'OSTEOMUSCULAR'):
#         row['grupo_doenca'] = 'OUTRAS CAUSAS'
#     else:
#         if (row['grupo_doenca'] != 'PSIQUIÁTRICA'):
#         row['grupo_doenca'] = 'OUTRAS CAUSAS'
#     else:
#         if (row['grupo_doenca'] != 'SEM CID-10'):
#         row['grupo_doenca'] = 'OUTRAS CAUSAS'

    return row
df_licencas_grupo_doenca = df_licencas_grupo_doenca.apply(colocaComoOutros, axis=1)

IndentationError: expected an indented block (<ipython-input-195-59e59040a9c0>, line 7)

In [ ]:
df_licencas_grupo_doenca['cib_copiado'] = df_licencas_grupo_doenca['cib_10'].copy()

In [ ]:
dicionario_doencas = df_licencas_grupo_doenca[['cib_10','grupo_doenca']].copy()

In [ ]:
dicionario_doencas['cib_10'].unique().shape

In [ ]:
dicionario_doencas.drop_duplicates('cib_10' ,keep = 'first', inplace = True)

In [ ]:
dicionario_doencas.shape

In [ ]:
df_licencas_grupo_doenca['mes'] = df_licencas_grupo_doenca['mes'].astype(int)

In [ ]:
lista_ordenacao_doenca = ['','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k','l']

In [ ]:
for mes in df_licencas_grupo_doenca['mes'].unique():
    
    df_aux_doenca = df_licencas_grupo_doenca[df_licencas_grupo_doenca['mes'] == mes].copy()
   
    df_agrupado_doenca = df_aux_doenca.groupby(['cib_10'], as_index = False).agg({'cib_copiado': 'count'}).copy()
       
    df_agrupado_doenca.rename(columns={"cib_copiado": lista_ordenacao_doenca[mes]}, inplace = True)
    
    dicionario_doencas = pd.merge(dicionario_doencas, df_agrupado_doenca, on = ['cib_10'], how = 'outer')
    
    dicionario_doencas.fillna(0, inplace = True)

In [ ]:
dicionario_doencas

# SALVANDOS OS DATA FRAMES

In [ ]:
writer = pd.ExcelWriter('Relatório de Atestados.xlsx', engine ='xlsxwriter')

### relatorio geral da quantidade de dias perdidos e da quantidade de licenças médicas gerais

df_atestados_meses.to_excel (writer, sheet_name = "relatorio_atestados", index = True, startrow = 1, startcol = 0)

workbook = writer.book

In [ ]:
df_atestados_meses.to_excel (writer, sheet_name = "relatorio_atestados", index = True, startrow = 1, startcol = 0)

In [ ]:
# tam_esp1 = df_atestados_meses.shape[0] + 4

In [ ]:
df_qtd_atestados_unidade.to_excel (writer, sheet_name = "relatorio_atestados", index = False, startrow = 8)

In [ ]:
# tam_esp2 = df_qtd_atestados_unidade.shape[0] + 4

In [ ]:
df_qtd_dias_perdidos_unidade.to_excel (writer, sheet_name = "relatorio_atestados", index = False, startrow = 59)

In [ ]:
# tam_esp3 = df_qtd_dias_perdidos_unidade.shape[0] + 4

In [ ]:
df_final_genero.to_excel(writer, sheet_name = "relatorio_atestados", index = True, startrow = 109)

In [ ]:
# tam_esp4 = df_final_genero.shape[0] + 4

In [ ]:
df_final_intervalos_idade.to_excel(writer, sheet_name = "relatorio_atestados", index = True, startrow = 115)

In [ ]:
# tam_esp5 = df_final_intervalos_idade.shape[0] + 4

In [ ]:
df_dicionario_funcoes.to_excel(writer, sheet_name = "relatorio_atestados", index = False, startrow = 125)

In [ ]:
# df_grupo_doenca.to_excel(writer, sheet_name = "relatorio_atestados", index = False, startrow = 182)

## Iniciando Formato da sheet com os cabeçalhos amarelos

In [ ]:
sheet_atestado = writer.sheets['relatorio_atestados']

format_atestado_cabecalho = workbook.add_format({
    'bold' : True,
    'align' : 'center',
    'valign': 'vcenter',
    'bg_color' : '#ffe599',
    'font_color': '#000000',
    'border' : 1
})

sheet_atestado.set_column('A:A',27)

sheet_atestado.merge_range("A1:M1", "LICENÇAS MÉDICAS - 2019", format_atestado_cabecalho)


sheet_atestado.merge_range("A8:M8", "QUANTIDADE DE LICENÇAS MÉDICAS POR UNIDADE", format_atestado_cabecalho)


In [ ]:
writer.save()